#Neural Nets


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
import tensorflow as tf
from keras import datasets, layers, models
tf.random.set_seed(123)

## Standard Feed Forward Neural Network:


In [ ]:
from sklearn.datasets import load_wine
wine = load_wine()
print(wine['DESCR'])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            10.6  30.0    19.5   3.3
    Ma

In [ ]:
from sklearn.preprocessing import  OneHotEncoder, MinMaxScaler

# Split the dataset
X = wine.data
y = wine.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# scale training data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert labels to OneHotEncoder
onehotencoder = OneHotEncoder(categories='auto')
y_train_encoded = onehotencoder.fit_transform(y_train.reshape(-1,1)).toarray()
y_test_encoded = onehotencoder.transform(y_test.reshape(-1,1)).toarray()

In [ ]:
y_train[:5]

array([2, 0, 1, 1, 2])

In [ ]:
y_train_encoded[:5]

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

create MLP architecture:

In [ ]:
mlp = models.Sequential()
mlp.add(layers.Dense(units=10, activation='relu')) # 'tanh'
# second hidden layer
mlp.add(layers.Dense(units=8, activation='relu'))
mlp.add(layers.Dense(units=3, activation='softmax'))

#### Training the FFMLP on the Training set

In [ ]:
mlp.compile(optimizer= 'adam', loss= 'categorical_crossentropy' , metrics=['accuracy'])
mlp.fit(X_train, y_train_encoded, batch_size=32, epochs=50,  verbose=1)


Epoch 1/50
5/5 [==============================] - 1s 4ms/step - loss: 1.1203 - accuracy: 0.3592
Epoch 2/50
5/5 [==============================] - 0s 3ms/step - loss: 1.1025 - accuracy: 0.3592
Epoch 3/50
5/5 [==============================] - 0s 3ms/step - loss: 1.0888 - accuracy: 0.3592
Epoch 4/50
5/5 [==============================] - 0s 4ms/step - loss: 1.0776 - accuracy: 0.3803
Epoch 5/50
5/5 [==============================] - 0s 3ms/step - loss: 1.0685 - accuracy: 0.4225
Epoch 6/50
5/5 [==============================] - 0s 3ms/step - loss: 1.0601 - accuracy: 0.4507
Epoch 7/50
5/5 [==============================] - 0s 3ms/step - loss: 1.0522 - accuracy: 0.4859
Epoch 8/50
5/5 [==============================] - 0s 3ms/step - loss: 1.0444 - accuracy: 0.5211
Epoch 9/50
5/5 [==============================] - 0s 3ms/step - loss: 1.0361 - accuracy: 0.5423
Epoch 10/50
5/5 [==============================] - 0s 3ms/step - loss: 1.0280 - accuracy: 0.5775
Epoch 11/50
5/5 [======================

#### Making the predictions and evaluating the model

In [ ]:
score, acc = mlp.evaluate(X_test, y_test_encoded, batch_size=32)

2/2 [==============================] - 0s 12ms/step - loss: 0.5580 - accuracy: 0.7778


In [ ]:
y_pred_nn = mlp.predict(X_test)
print(y_pred_nn[-5:,])
predicted_class = np.argmax(y_pred_nn, axis=1)
predicted_class[-5:,]

2/2 [==============================] - 0s 5ms/step
[[0.69009346 0.21626069 0.09364584]
 [0.0545746  0.49036184 0.45506364]
 [0.10535929 0.5997928  0.29484797]
 [0.08107867 0.42128858 0.49763274]
 [0.14021944 0.5673996  0.2923809 ]]


array([0, 1, 1, 2, 1])

In [ ]:
acc = metrics.accuracy_score(predicted_class, y_test)
acc

0.7777777777777778

## Radial Basis Function Neural Network

RBF models the data using smooth transitioning circular shapes instead of sharp cut-off circles. Additionally, RBF gives information about the confidence rate of prediction which the K-means Clustering algorithm can’t.

![](https://drive.google.com/uc?export=view&id=1iOOQthoNifkPxg2Sg8z5YCAt98RoSamJ)
 ![](https://drive.google.com/uc?export=view&id=1KJre0KOkin4Hu_WVdoP8bg8FLUqMpe63)

 [source](https://towardsdatascience.com/most-effective-way-to-implement-radial-basis-function-neural-network-for-classification-problem-33c467803319)


RBF code adapted from [here](https://www.madrasresearch.org/post/radial-basis-functions-neural-networks).

In [ ]:
from sklearn.cluster import KMeans
import math

determine centers of the radial base neurons using KMeans:

In [ ]:
k_center=10
kmeans = KMeans(n_clusters=k_center, init="k-means++", max_iter=100, random_state=123)
kmeans.fit(X_train)
center = kmeans.cluster_centers_
center.shape # (k_center, X_train_c.shape[1])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


(10, 13)

In [ ]:
# @title code showing how to get cluster standard deviations (optional)
labels = kmeans.labels_
# Create a dictionary to store standard deviations
cluster_stds = {}

# Iterate over unique cluster labels
for label in np.unique(labels):
  # Select data points belonging to the current cluster
  cluster_data = X_train[labels == label]
  # Calculate standard deviation for each feature dimension
  cluster_stds[label] = np.std(cluster_data, axis=0)

determine σ:

In [ ]:
max=0
for i in range(k_center):
  for j in range(k_center):
    d=np.linalg.norm(center[i] - center[j]) # distance norm betwen two hidden neurons
    if(d>max):
      max = d
d = max
sigma = d/math.sqrt(2*k_center)

determine the output of radial basis hidden neurons:

In [ ]:
h = np.empty((X_train.shape[0] , k_center), dtype=float)

for i in range(X_train.shape[0]):
  for j in range(k_center):
    dist = np.linalg.norm(X_train[i] - center[j])
    h[i][j] = math.exp(-math.pow(dist,2)/2*math.pow(sigma,2))

weight matrix W drawn from Least Square Method to train the network:

In [ ]:
w = np.dot(np.linalg.pinv(h), y_train_encoded)

Matrix phi for test data:

In [ ]:
h_test = np.empty((X_test.shape[0] , k_center), dtype=float)

for i in range(X_test.shape[0]):
  for j in range(k_center):
    dist = np.linalg.norm(X_test[i] - center[j])
    h_test[i][j] = math.exp(- math.pow(dist,2)/2*math.pow(sigma,2))

Prediction Performance:

In [ ]:
predictions = np.dot(h_test, w)
y_pred = predictions.argmax(axis=1)

acc = metrics.accuracy_score(y_pred, y_test)
acc

0.9722222222222222

## Extreme Machine Neural Networks

Extreme Learning Machines (ELM) are a type of single-hidden-layer feedforward neural network known for their fast training speed and good generalization performance.


*   ELM uses randomly generated weights for the connections between the input layer and the hidden layer. These weights remain fixed during the training process.
*   The connections between the hidden layer and the output layer are determined using a linear regression technique. This avoids the iterative optimization (backpropagation).

 ![](https://drive.google.com/uc?export=view&id=1eL_GxSMzaQqati-jG8wxci2o3T2mhgAo)



initialize our input weights and biases randomly, drawn from a Gaussian distribution.

In [ ]:
np.random.seed(123)

input_size = X_train.shape[1]
hidden_size = 10
input_weights = np.random.normal(size=[input_size,hidden_size])
biases = np.random.normal(size=[hidden_size])

Compute hidden states, find output weights using the [least square method](https://numpy.org/doc/stable/reference/generated/numpy.linalg.pinv.html):

In [ ]:
def hidden_nodes(X): # hh_j = activation(W_ij*X_i + b_j)
    G = np.dot(X, input_weights)
    G = G + biases
    H = tf.keras.activations.relu(G)
    return H
H =hidden_nodes(X_train)
output_weights = np.dot(np.linalg.pinv(H), y_train_encoded) # pinv: Compute the (Moore-Penrose) pseudo-inverse of a matrix.
output_weights

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.02752758e-17, -9.02816775e-17, -3.31905010e-17],
       [ 1.40662212e-01, -2.45062935e-01, -5.37764401e-02],
       [-2.08267994e-01,  4.46220510e-01,  1.72842284e-02],
       [ 2.66311777e-01, -1.69744218e-01, -3.01234496e-01],
       [ 8.88887790e-02, -1.17513579e-01,  2.03637726e-01],
       [ 4.91955465e-02, -5.20676176e-01,  3.74616991e-01],
       [-2.48891147e-01,  2.70500446e-01,  1.80169598e-01],
       [ 6.13586384e-02,  7.47847674e-02, -1.20936251e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

Predict on a test set:

In [ ]:
def predict(X):
    out = hidden_nodes(X)
    out = np.dot(out, output_weights)
    return out

prediction = predict(X_test)
y_pred = prediction.argmax(axis=1)

acc = metrics.accuracy_score(y_test, y_pred)
acc

0.6388888888888888

## Generalized Regression Neural Networks

Please refer to this GitHub repository: https://github.com/federhub/pyGRNN